# get_frequently_mutated Final 

In [1]:
def get_frequently_mutated(cancer_type, cutoff=.1):  
    """take cancer type to import cptac and find the frequently mutated genes in total tumors compared to the cutoff.
        
        Parameters:
        cancer_type (string): type of  cancer 
        cutoff (float): used as comparison to determine status of gene mutation frequency
        
        Returns:
        freq_mutated_df (pd.DataFrame): DataFrame of frequently mutated genes passing the cutoff
            and Total_Mutated (mutated genes / total tumors), percent Truncated, and percent Missence
        
        There are many types of mutations catagorized into the columns Truncated and Missence. 
        The Truncated column includes: Frame_Shift_Del, Frame_Shift_Ins, Splice_Site, Nonsense_Mutation, Nonstop_Mutation
        The Missence column includes: In_Frame_Del, In_Frame_Ins, Missense_Mutation
        These columns count multiple mutations of one gene in the same sample, so percentages in the last two columns may 
        exceed the Total_Mutated column(which only counts if the gene was mutated once)"""    
    
    # import CPTAC and pandas
    
    import pandas as pd
    colon = False
    if cancer_type == "endometrial" or cancer_type == "Endometrial":
        import cptac.endometrial as cptac
        
    elif cancer_type == "colon" or cancer_type == "Colon":
        import cptac.colon as cptac
        colon = True
        
    elif cancer_type == "ovarian" or cancer_type == "Ovarian":
        import cptac.ovarian as cptac
    
    else:
        str_cancer_options = '\n' + 'Options: endometrial, ovarian, colon'
        print("Please enter a valid cancer type.", str_cancer_options)
        return 0
    
    # get data frames
    somatic_mutations = cptac.get_mutations()
    proteomics = cptac.get_proteomics()
    sample_status_map = cptac.get_sample_status_map()
    merged_mutations = somatic_mutations.join(sample_status_map, how="left") 
    
    # standardize mutation names 
    if colon == True:
        mutation_equivalents = {'frameshift substitution': 'Frame_Shift_Del' , 'frameshift deletion': 'Frame_Shift_Del', 
            'frameshift insertion': 'Frame_Shift_Ins', 'stopgain': 'Nonsense_Mutation ', 'stoploss':'Nonstop_Mutation',
            'nonsynonymous SNV': 'Missense_Mutation','nonframeshift insertion': 'In_Frame_Ins',
            'nonframeshift deletion': 'In_Frame_Del', 'nonframeshift substitution': 'Missense_Mutation'}
        merged_mutations = merged_mutations.replace(to_replace = mutation_equivalents)
        
    # get list of unique genes
    unique_genes = somatic_mutations['Gene'].unique()
    
    # get total tumors/patients
    sample_status_series = sample_status_map.value_counts()
    total_tumor_patients = sample_status_series[0]
        
    # find frequently mutated genes and their total mutated fraction. Create lists for frequently mutated genes and fraction.
    freq_mut = []
    total_fraction_mutated = []
    for gene in unique_genes:
        gene_mutated = merged_mutations.loc[merged_mutations['Gene'] == gene]
        gene_mutated = gene_mutated.index.unique()
        num_gene_mutated = len(gene_mutated)
        fraction = (num_gene_mutated / total_tumor_patients)
        if fraction > cutoff:
            freq_mut.append(gene)
            total_fraction_mutated.append(fraction)
    
    # find truncated fraction
    truncated = []
    for gene in freq_mut:
        gene_mutated = merged_mutations.loc[merged_mutations['Gene'] == gene]
        truncated_df = gene_mutated.loc[(gene_mutated['Mutation'] != 'In_Frame_Del') & 
            (gene_mutated['Mutation'] != 'In_Frame_Ins') & (gene_mutated['Mutation'] != 'Missense_Mutation')] 
        samples_trunc = truncated_df.index.unique()
        num_trunc_mut = len(samples_trunc)
        fraction_trunc = (num_trunc_mut / total_tumor_patients)
        truncated.append(fraction_trunc)

    # find missence fraction 
    missence = []
    for gene in freq_mut:
        gene_mutated = merged_mutations.loc[merged_mutations['Gene'] == gene]
        missence_mutations = gene_mutated.loc[(gene_mutated['Mutation'] == 'In_Frame_Ins') | (gene_mutated['Mutation'] == 'In_Frame_Del') | (gene_mutated['Mutation'] == 'Missense_Mutation')]
        samples_miss = missence_mutations.index.unique()
        num_miss_mut = len(samples_miss)
        fraction_miss = (num_miss_mut / total_tumor_patients)
        missence.append(fraction_miss)
        
    # create dataframe
    merged_lists = list(zip(freq_mut, total_fraction_mutated, truncated, missence))
    freq_mutated_df = pd.DataFrame(merged_lists, columns =['Gene', 'Fraction_Mutated', 'Truncation', 'Missence'])
                   
    return freq_mutated_df

Test Endometrial

In [2]:
endo_freq_mutated_df = get_frequently_mutated("endometrial", .25)

Welcome to the cptac data service package. Available datasets may be
viewed using cptac.list_data(). In order to access a specific data
set, import a cptac subfolder using either 'import cptac.dataset' or
'from cptac import dataset'.
******
Version: 0.4.1
******
You have loaded the cptac endometrial dataset. To view available
dataframes, use cptac.endometrial.list_data(). To view available
functions for accessing and manipulating the dataframes, use
cptac.endometrial.list_api().
endometrial data version: 2.1

Loading Dictionary...
Loading cptac endometrial data:
Loading acetylproteomics data...
Loading clinical data...
Loading CNA data...
Loading miRNA data...
Loading phosphoproteomics_gene data...
Loading phosphoproteomics_site data...
Loading proteomics data...
Loading somatic data...
Loading somatic_binary data...
Loading transcriptomics_circular data...
Loading transcriptomics_linear data...

 ******PLEASE READ******
CPTAC is a community resource project and data are made available

In [26]:
endo_freq_mutated_df


,Gene,Fraction_Mutated,Truncation,Missence
0,ARID1A,0.452632,0.400000,0.136842
1,PIK3CA,0.494737,0.010526,0.484211
2,PTEN,0.789474,0.568421,0.463158
3,CTCF,0.284211,0.242105,0.094737
4,KRAS,0.326316,0.000000,0.326316
5,PIK3R1,0.389474,0.189474,0.231579
6,CTNNB1,0.305263,0.000000,0.305263


In [31]:
ctcf = 27/95
pten = 75/95
arid1a = 43/95

In [38]:
import pandas as pd
check_getter(endo_freq_mutated_df, 'name', (7, 4), ['Gene', 'Fraction_Mutated', 'Truncation', 'Missence'],
            ((3,1),(2,1),(0,1)), (ctcf, pten, arid1a))

Dataframe did not have a "name" attribute.


False

In [23]:
def check_getter(df, exp_name, exp_dim, exp_headers, coordinates, values): 
    """Test a dataframe's name, dimensions and headers, and three test values, then print whether it passed the test.
    Parameters
    df: the dataframe gotten by the getter we are testing
    exp_name: string containing the expected name of the dataframe gotten by the getter we're testing
    exp_dim: a tuple containing the expected dimensions of the dataframe, in the format (rows, columns)
    exp_headers: if the dataframe has up to 20 columns, all of the headers for the dataframe, in order. If it has more than 20 columns, then a list containing the first ten and last ten headers, in order.
    coordinates: a tuple with three elements, each element being a tuple with two elements, the first element being the int index of the row of a test value, and the second element being the int index of the column of a test value
    values: a tuple with three elements, each element being the expected value of the test value corresponding to the coordinates at the same index in the coordinates parameter 
    Returns
    bool indicating if the dataframe had the correct data.
    """
    PASS = True

    # Check that df is a dataframe, not None or something else.
    if not check_returned_is_df(df):
        return False # End test, because other tests will be useless.

    # Check name
    if not check_df_name(df, exp_name):
        PASS = False

    # Check dimensions
    if not check_df_shape(df, exp_dim):
        PASS = False

    # Check headers
    act_headers_all = list(df.columns.values)
    if len(df.columns.values) <= 20:
        act_headers = act_headers_all
    else:
        act_headers = act_headers_all[:10] + act_headers_all[-10:]

    if len(exp_headers) != len(act_headers):
        print("Unexpected number of test headers in dataframe. Expected number of headers: {}. You passed {} headers.\n".format(len(act_headers), len(exp_headers)))
        PASS = False
    else:
        for i, header in enumerate(exp_headers):
            if header != act_headers[i]:
                print("Dataframe header did not match expected value.\n\tExpected: {}\n\tActual: {}\n".format(header, act_headers[i]))
                PASS = False

    # Check test values
    act_values = [
        df.iloc[coordinates[0][0], coordinates[0][1]],
        df.iloc[coordinates[1][0], coordinates[1][1]],
        df.iloc[coordinates[2][0], coordinates[2][1]]]

    for i, value in enumerate(values):
        if act_values[i] != value:
            print("Dataframe value did not match expected value.\n\tColumn: {}\n\tIndex: {}\n\tExpected: {}\n\tActual: {}\n".format(df.columns.values[coordinates[i][1]], df.index.values[coordinates[i][0]], value, act_values[i]))
            PASS = False

    # Return whether the dataframe passed the test
    return PASS

In [24]:
def print_test_result(PASS):
    """Prints the result of a test, based on a bool.
    Parameters:
    PASS (bool): Whether or not the test passed.
    """
    if PASS:
        print('\tPASS')
    else:
        print('\tFAIL\n')

def check_returned_is_df(returned):
    """Checks that an object is a dataframe. Prints a specific message if it's actually None, or a general message if it's something else.
    Parameters:
    returned: The object to test
    Returns:
    bool: Indicates whether the object was a dataframe.
    """
    if returned is None:
        print("Function under test returned None.")
        return False
    
    if not isinstance(returned, pd.core.frame.DataFrame):
        print("Returned object was not a dataframe. Type of object: {}".format(type(returned)))
        return False
    return True

def check_df_name(df, expected_name):
    """Checks that a dataframe has a "name" attribute, and that it has the proper value.
    Parameters:
    df (pandas.core.frame.DataFrame): The dataframe to test.
    expected_name (str): The expected name of the dataframe.
    Returns:
    bool: Whether the dataframe had a name, and it was the correct name.
    """
    # Check that the dataframe has a name
    if not hasattr(df, 'name'):
        print('Dataframe did not have a "name" attribute.')
        return False

    # Check that the dataframe has the correct name
    if df.name != expected_name:
        print("Dataframe had incorrect name.\n\tExpected: {}\n\tActual: {}".format(expected_name, df.name))
        return False
    return True

def check_df_shape(df, exp_shape):
    """Checks that a dataframe has the proper shape.
    Parameters:
    df (pandas.core.frame.DataFrame): The dataframe to test.
    exp_shape (tuple): A tuple with two elements. First element is expected number of rows, second is expected number of columns.
    Returns:
    bool: Indicates whether the dataframe had the proper shape.
    """
    act_shape = df.shape
    if exp_shape != act_shape:
        print("Dataframe dimensions did not match expected values.\n\tExpected: {}\n\tActual: {}\n".format(exp_shape, act_shape))
        return False
    return True

In [ ]:
check_getter()

Test Colon

In [9]:
colon_freq_mutated_df = get_frequently_mutated("colon", .25)

You have loaded the cptac colon dataset. To view available dataframes,
use cptac.colon.list_data(). To view available functions for accessing
and manipulating the dataframes, use cptac.colon.list_api().
colon data version: Most recent release

Loading cptac colon data:
Loading clinical data...
Loading miRNA data...
Loading mutation data...
Loading mutation_binary data...
Loading phosphoproteomics_normal data...
Loading phosphoproteomics_tumor data...
Loading proteomics_normal data...
Loading proteomics_tumor data...
Loading transcriptomics data...
['Missense_Mutation' 'Frame_Shift_Del' 'Frame_Shift_Ins'
 'Nonsense_Mutation ' 'In_Frame_Ins' 'In_Frame_Del' 'Nonstop_Mutation']
total_tumor_patients 110


In [10]:
colon_freq_mutated_df

,Gene,Percent Mutated,Truncated,Missence
0,APC,0.745455,3.136364,0.381818
1,TP53,0.509091,2.145455,4.845455
2,MUC16,0.354545,0.100000,0.627273
3,FAT3,0.254545,0.036364,0.381818
4,SYNE1,0.263636,0.281818,1.018182
5,TTN,0.545455,1.409091,8.318182
6,KRAS,0.318182,0.000000,0.636364
7,OBSCN,0.263636,0.190909,1.127273
8,CCDC168,0.263636,0.263636,0.336364


Test Ovarian

In [11]:
ovarian_freq_mutated_df = get_frequently_mutated("ovarian", .25)

You have loaded the cptac ovarian dataset. To view available
dataframes, use cptac.ovarian.list_data(). To view available functions
for accessing and manipulating the dataframes, use
cptac.ovarian.list_api().
ovarian data version: Most recent release

Loading cptac ovarian data:
Loading clinical data...
Loading cnv data...
Loading phosphoproteomics data...
Loading proteomics data...
Loading somatic_38 data...
Loading transcriptomics data...
Loading treatment data...

 ******PLEASE READ******
CPTAC is a community resource project and data are made available
rapidly after generation for community research use. The embargo
allows exploring and utilizing the data, but the data may not be in a
publication until June 1, 2019. Please see
https://proteomics.cancer.gov/data-portal/about/data-use-agreement or
enter embargo() to open the webpage for more details.
total_tumor_patients 111


In [12]:
ovarian_freq_mutated_df

,Gene,Percent Mutated,Truncated,Missence
0,TTN,0.279279,0.252252,0.468468
1,TP53,0.693694,0.243243,0.468468
2,MUC4,0.306306,0.153153,0.432432
